In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib as mpl
from skimage.segmentation import flood_fill
from skimage.io import imread
from skimage.color import rgb2gray
from skimage.transform import resize
from skimage.filters import threshold_local

In [ ]:
def box(height, width):
    arr = np.zeros((height, width), dtype=np.uint8)
    # walls
    arr[0] = 1
    arr[-1] = 1
    arr[..., 0] = 1
    arr[..., -1] = 1
    arr[0, 1] = 2
    arr[-1, -2] = 2
    return arr

b = box(6, 6)
b

In [ ]:
mpl.cm.inferno.resampled(3)

In [ ]:
palette = mpl.cm.inferno.resampled(3).colors
labels = ["0: unfilled", "1: wall", "2: passage"]

def show(arr):
    plt.figure(figsize=(9, 9))
    im = plt.imshow(palette[arr])
    patches = [mpatches.Patch(color=c, label=l) for c, l in zip(palette, labels)]
    plt.legend(handles=patches, bbox_to_anchor=(1.1, 1), loc=2, borderaxespad=0)
    plt.show()

show(b)

In [ ]:
np.where(b == 2)

In [ ]:
start = next(zip(*np.where(b == 2)))
start

In [ ]:
np.where(b == 0)

In [ ]:
np.swapaxes(np.where(b == 0), 0, 1)

In [ ]:
a = box(30,30)
show(a)

In [ ]:
def maze1(arr):
    unfilled = np.swapaxes(np.where(arr == 0), 0, 1)
    np.random.shuffle(unfilled)

    start = next(zip(*np.where(arr == 2)))
    arr = np.copy(arr)
    arr[arr == 2] = 0

    for lc in unfilled:
        lc = tuple(lc)
        arr[lc] = 1
        t = flood_fill(arr, start, 1)
        if np.any(t == 0):
            arr[lc] = 0

    arr[arr == 0] = 2
    return arr

show(maze1(a))

In [ ]:
def maze2(arr):
    unfilled = np.swapaxes(np.where(arr == 0), 0, 1)
    np.random.shuffle(unfilled)

    start = next(zip(*np.where(arr == 2)))
    arr = np.copy(arr)
    arr[arr == 2] = 0

    for lc in unfilled:
        lc = tuple(lc)
        arr[lc] = 1
        t = flood_fill(arr, start, 1, connectivity=1)
        if np.any(t == 0):
            arr[lc] = 0

    arr[arr == 0] = 2
    return arr

show(maze2(a))

In [ ]:
neighbours = np.array([
    [0, 1, 0],
    [1, 0, 1],
    [0, 1, 0]], dtype=np.uint8)

def maze3(arr):
    unfilled = np.swapaxes(np.where(arr == 0), 0, 1)
    np.random.shuffle(unfilled)

    start = next(zip(*np.where(arr == 2)))
    arr = np.copy(arr)
    arr[arr == 2] = 0

    for lc in unfilled:
        lc = tuple(lc)
        
        y, x = lc
        if np.sum(neighbours * arr[y-1:y+2, x-1:x+2]) > 2:
            continue
        
        arr[lc] = 1
        t = flood_fill(arr, start, 1, connectivity=1)
        if np.any(t == 0):
            arr[lc] = 0

    arr[arr == 0] = 2
    return arr

m = maze3(a)
show(m)

In [ ]:
# spoiler
t = flood_fill(m, (0,0), 0)
show(t)

In [ ]:
c = box(31, 31)
c[::2, ::2] = 1
c[1::2, 1::2] = 2
show(c)

In [ ]:
sm = maze3(c)
show(sm)

In [ ]:
# spoiler
t = flood_fill(sm, (0,0), 0)
show(t)

In [ ]:
brain = imread('brain-coral.jpg')
plt.figure(figsize=(9, 9))
im = plt.imshow(brain)

In [ ]:
crop_brain = rgb2gray(brain[-500:,200:1000])
plt.figure(figsize=(15, 9))
im = plt.imshow(crop_brain, plt.cm.gray)
cb = plt.colorbar()

In [ ]:
small_brain = resize(crop_brain, (50, 80))
binary_brain = small_brain > threshold_local(small_brain, 15, 'mean')
plt.figure(figsize=(9, 9))
im = plt.imshow(binary_brain, plt.cm.gray)

In [ ]:
d = box(52, 82)
insert_brain = (binary_brain + 2) % 3 
d[1:-1, 1:-1] = insert_brain
show(d)

In [ ]:
bm = maze2(d)  # maze2 is better: don't need lots of extra dead-ends
show(bm)

In [ ]:
# spoiler
t = flood_fill(bm, (0,0), 0)
show(t)

# Thank you!

https://excess.org/cpu

https://github.com/wardi/cpu/maze